# Déployement sur Azure Machine Learning du modèle final

<img src="./img/Microsoft-Azure-Logo.png" style="max-height: 200px; margin-left:auto; margin-right:auto; display: block;">

Ce notebook permet le déploiement du modèle avancé sur Azure Machine Learning.

Pour ça il est nécessaire de créer une ressource Azure Machine Learning sur le <a href="https://portal.azure.com/#create/Microsoft.MachineLearningServices">"portal.azure.com"</a> et de créer les variables d'environnement de l'abonnement ID (SUBSCRIPTION_ML), le nom du gorupe de ressource (RESOURCE_GROUP_ML), et le nom de l'espace de travail Azure Machine Learning fraichement créé (WORKSPACE_ML).

In [ ]:
import sys, os
# import mlflow
# import mlflow.azureml

import azureml.core
from azureml.core import Workspace, Environment
from azureml.core import Model

## Connexion à l'espace de travail Azure Machine Learning

In [2]:
subscription_id = os.environ["SUBSCRIPTION_ML"]
resource_group = os.environ["RESOURCE_GROUP_ML"]
workspace_name = os.environ["WORKSPACE_ML"]

In [3]:
ws = Workspace(subscription_id=subscription_id,
               resource_group=resource_group,
               workspace_name=workspace_name)

## Test du modèle sauvegardé

In [5]:
import tensorflow as tf
from keras.models import load_model
import joblib
import re
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

path_to_model="./data/final_model"

tokenise = joblib.load(path_to_model+"/tokenizer.pkl")
model = load_model(path_to_model+"/w2v_model")

def preprocess_data(data : list):
    return pad_sequences(tokenise.texts_to_sequences([cleanning_data(tweet) for tweet in data]),maxlen=300)

def cleanning_data(sentence):
    sentence += " "
    sentence = re.sub('\n+', ' ', sentence.lower())
    # special caractère
    sentence = re.sub('[éèêë]','e',sentence)
    sentence = re.sub('[ïîì]','i',sentence)
    sentence = re.sub('[àâä]','a',sentence)
    sentence = re.sub('[ôö]','o',sentence) 
    sentence = re.sub('[üûù]','u',sentence)
    sentence = re.sub('[ç]','c',sentence)
    # personal_clean
    sentence = clean_escape(sentence) # usless ?
    sentence = clean_url(sentence)
    sentence = clean_smile(sentence)
    sentence = re.sub(r'[^a-z@#\'_ ]+',' ',sentence)
    sentence = clean_uniuque_char(sentence)
    sentence = re.sub(' {2,}', ' ', sentence)
    return sentence

def clean_uniuque_char(sentence):
    sentence = " ".join([v for v in sentence.split(" ") if ((len(v) > 1) or (not v.isalpha()))])
    return sentence

def clean_escape(sentence):
    sentence = re.sub('&amp;','&',sentence)
    sentence = re.sub('&quot;','\"',sentence)
    sentence = re.sub('&gt;','>',sentence)
    sentence = re.sub('&lt;','<',sentence)
    return sentence

def clean_smile(sentence):
    sentence = re.sub(';\)','emote_clin_doeil ',sentence)
    sentence = re.sub(':\)|=\)','emote_sourire ',sentence)
    sentence = re.sub('x\)','emote_cross_smile ',sentence)
    sentence = re.sub('x{1,}[Dd]{1,}[^A-Za-z0-9]','emote_mdr ',sentence)
    sentence = re.sub(';[pP]{1,}[^A-Za-z0-9]|:[pP]{1,}[^A-Za-z0-9]|x[pP]{1,}[^A-Za-z0-9]|=[pP]{1,}[^A-Za-z0-9]',
                      'emote_tire_langue ',sentence)
    sentence = re.sub('[oO]_[oO]','emote_tres_surpris',sentence)
    sentence = re.sub('[Xx]{1,}[Oo]{1,}[^A-Za-z0-9]','emote_ko ',sentence)
    sentence = re.sub('[bB]\)[^A-Za-z0-9]','emote_lunette ',sentence)
    sentence = re.sub('[=:;][oO]{1,}[^A-Za-z0-9]','emote_surpris ',sentence)
    sentence = re.sub('[=:;][S]{1,}[^A-Za-z0-9]','emote_genant ',sentence)
    sentence = re.sub('[:;=][dD]{1,}[^A-Za-z0-9]','emote_gros_sourire ',sentence)
    sentence = re.sub('[:;=][$]{1,}[^A-Za-z0-9]','emote_genant_discret ',sentence)
    sentence = re.sub('[:;=]\|[^A-Za-z0-9]','emote_neutre ',sentence)
    sentence = re.sub('[:;=]/[^A-Za-z0-9]','emote_deception ',sentence)
    sentence = re.sub('[:;=]\([^A-Za-z0-9]','emote_insatisfait ',sentence)
    sentence = re.sub('\*[-_]\*[^A-Za-z0-9]','emote_magnifique ',sentence)
    sentence = re.sub('\^[-_]{0,1}\^[^A-Za-z0-9]','emote_joyeux ',sentence)
    sentence = re.sub('--\'[^A-Za-z0-9]','emote_enerve ',sentence)
    return sentence

def clean_url(sentence):
    sentence = re.sub('http[s]*://[0-9a-zA-Z-_.]*.[a-z]{0,3}/[0-9a-zA-Z-_./]*', '', sentence)
    return sentence

In [6]:
data = preprocess_data(["realy ?","I love banana!","I hate banana!"])

In [14]:
y_pred_proba = model.predict(data)
y_pred = np.argmax(model.predict(data),axis=1)

In [28]:
print(y_pred)
print("---------")
print(y_pred_proba)

[0 1 0]
---------
[[0.5611326  0.43886733]
 [0.05260534 0.9473947 ]
 [0.95672864 0.04327139]]


## Upload du modèle Keras et du Tokenizer sur l'espace de travail

In [30]:

tokenizer_azure = Model.register(model_path=path_to_model+"/tokenizer.pkl",
                       model_name="tokenizer_keras",
                       tags={'area': "NLP", 'type': "tokenizer"},
                       description="Tokenizer entrainer avec le jeu de données final.",
                       workspace=ws)

Registering model tokenizer_keras


In [32]:
w2v_model_azure = Model.register(model_path=path_to_model+"/w2v_model",
                       model_name="keras_w2v_model",
                       tags={'area': "NLP", 'type': "deep_learning"},
                       description="Word2Vec LSTM modèle keras entrainé",
                       workspace=ws)

Registering model keras_w2v_model


In [ ]:
# Permet de récupérer l'instance cloud du modèle si il existe déjà dans l'espace de travail
# Il est également possible d'indiquer la version avec l'argument "version=2"

# tokenizer_azure = Model(ws, 'tokenizer_keras')
# w2v_model_azure = Model(ws, 'keras_w2v_model')

## Création du script d'entrée de l'inférence

In [33]:
import os

source_directory = "./advanced_model"

os.makedirs(source_directory, exist_ok=True)
os.makedirs(os.path.join(source_directory, "x/y"), exist_ok=True)

In [53]:
%%writefile advanced_model/x/y/score.py
import joblib
import json
import numpy as np
import os
import re

import tensorflow as tf

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, Dropout

from azureml.core.model import Model


def init():
    global model
    global tokenise

    tokenizer_path = Model.get_model_path(model_name = 'tokenizer_keras')
    keras_path = Model.get_model_path(model_name = 'keras_w2v_model')

    try:
        tokenise = joblib.load(tokenizer_path)
    except:
        print("tokenizer not work")
        
    try: 
        model = load_model(keras_path)
    except:
        print("Keras model not working")

def run(request):
    try:
        req = json.loads(request)
        data = req["data"]
        method = req["method"] 
        if type(data) is list:
            data = preprocess_data(data)
        else:
            return "Please enter list of tweet as type of \"list\""
            
        if str(method) == "prediction":
            dic = {1:"good",0:"bad"}
            result = np.argmax(model.predict(data),axis=1)
            return [dic[y] for y in result]
            
        elif str(method) == "score":
            result = model.predict(data)
            return result[:,1].tolist()
        
        elif str(method) == "prediction_with_neutral":
            result = model.predict(data)
            result = result[:,1]
            return [predict_with_neutral(y) for y in result]
        else:
            return """method aviable : \"prediction\",\"score\",\"prediction_with_neutral\"
 - prediction return labels sentiment (bad/good)
 - score return score of sentiment 0(bad) to 1(good)
 - prediction_with_neutral return labels sentiment with label neutral"""
            
    except Exception as e:
        error = str(e)
        return error
    
def predict_with_neutral(score):
    if score >= 0.6:
        return "good"
    elif score <= 0.4:
        return "bad"
    else:
        return "neutral"

def preprocess_data(data : list):
    return pad_sequences(tokenise.texts_to_sequences([cleanning_data(tweet) for tweet in data]),maxlen=300)

def cleanning_data(sentence):
    sentence += " "
    sentence = re.sub('\n+', ' ', sentence.lower())
    # special caractère
    sentence = re.sub('[éèêë]','e',sentence)
    sentence = re.sub('[ïîì]','i',sentence)
    sentence = re.sub('[àâä]','a',sentence)
    sentence = re.sub('[ôö]','o',sentence) 
    sentence = re.sub('[üûù]','u',sentence)
    sentence = re.sub('[ç]','c',sentence)
    # personal_clean
    sentence = clean_escape(sentence) # usless ?
    sentence = clean_url(sentence)
    sentence = clean_smile(sentence)
    sentence = re.sub(r'[^a-z@#\'_ ]+',' ',sentence)
    sentence = clean_uniuque_char(sentence)
    sentence = re.sub(' {2,}', ' ', sentence)
    return sentence

def clean_uniuque_char(sentence):
    sentence = " ".join([v for v in sentence.split(" ") if ((len(v) > 1) or (not v.isalpha()))])
    return sentence

def clean_escape(sentence):
    sentence = re.sub('&amp;','&',sentence)
    sentence = re.sub('&quot;','\"',sentence)
    sentence = re.sub('&gt;','>',sentence)
    sentence = re.sub('&lt;','<',sentence)
    return sentence

def clean_smile(sentence):
    sentence = re.sub(';\)','emote_clin_doeil ',sentence)
    sentence = re.sub(':\)|=\)','emote_sourire ',sentence)
    sentence = re.sub('x\)','emote_cross_smile ',sentence)
    sentence = re.sub('x{1,}[Dd]{1,}[^A-Za-z0-9]','emote_mdr ',sentence)
    sentence = re.sub(';[pP]{1,}[^A-Za-z0-9]|:[pP]{1,}[^A-Za-z0-9]|x[pP]{1,}[^A-Za-z0-9]|=[pP]{1,}[^A-Za-z0-9]',
                      'emote_tire_langue ',sentence)
    sentence = re.sub('[oO]_[oO]','emote_tres_surpris',sentence)
    sentence = re.sub('[Xx]{1,}[Oo]{1,}[^A-Za-z0-9]','emote_ko ',sentence)
    sentence = re.sub('[bB]\)[^A-Za-z0-9]','emote_lunette ',sentence)
    sentence = re.sub('[=:;][oO]{1,}[^A-Za-z0-9]','emote_surpris ',sentence)
    sentence = re.sub('[=:;][S]{1,}[^A-Za-z0-9]','emote_genant ',sentence)
    sentence = re.sub('[:;=][dD]{1,}[^A-Za-z0-9]','emote_gros_sourire ',sentence)
    sentence = re.sub('[:;=][$]{1,}[^A-Za-z0-9]','emote_genant_discret ',sentence)
    sentence = re.sub('[:;=]\|[^A-Za-z0-9]','emote_neutre ',sentence)
    sentence = re.sub('[:;=]/[^A-Za-z0-9]','emote_deception ',sentence)
    sentence = re.sub('[:;=]\([^A-Za-z0-9]','emote_insatisfait ',sentence)
    sentence = re.sub('\*[-_]\*[^A-Za-z0-9]','emote_magnifique ',sentence)
    sentence = re.sub('\^[-_]{0,1}\^[^A-Za-z0-9]','emote_joyeux ',sentence)
    sentence = re.sub('--\'[^A-Za-z0-9]','emote_enerve ',sentence)
    return sentence

def clean_url(sentence):
    sentence = re.sub('http[s]*://[0-9a-zA-Z-_.]*.[a-z]{0,3}/[0-9a-zA-Z-_./]*', '', sentence)
    return sentence

Overwriting advanced_model/x/y/score.py


## Création de l'inférence & environnement

In [54]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice

env_tf_sk = Environment('tensorflow-scikit')
env_tf_sk.python.conda_dependencies.add_pip_package("joblib")
env_tf_sk.python.conda_dependencies.add_pip_package("scikit-learn")
env_tf_sk.python.conda_dependencies.add_pip_package("tensorflow")
env_tf_sk.python.conda_dependencies.add_pip_package("azureml-core")


inference_config = InferenceConfig(source_directory=source_directory,
                                   entry_script="x/y/score.py",
                                   environment=env_tf_sk)

## Configuration ACI et déploiement

In [56]:
# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 4,auth_enabled=True)

# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(
    workspace = ws,
    name = "tweet-sentiment-analysis",
    models = [tokenizer_azure,w2v_model_azure],
    inference_config = inference_config,
    deployment_config = deployment_config)

service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-11-24 21:31:19+01:00 Creating Container Registry if not exists.
2021-11-24 21:31:19+01:00 Registering the environment.
2021-11-24 21:31:21+01:00 Use the existing image.
2021-11-24 21:31:21+01:00 Generating deployment configuration.
2021-11-24 21:31:23+01:00 Submitting deployment to compute.
2021-11-24 21:31:28+01:00 Checking the status of deployment tweet-sentiment-analysis..
2021-11-24 21:33:29+01:00 Checking the status of inference endpoint tweet-sentiment-analysis.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [57]:
primary, secondary = service.get_keys()
print(primary)
print(service.scoring_uri)

aocNmbc0clYZ1ToPtcTeJl9UPZlkJMDV
http://212175d7-6cbf-47be-a1d7-6f3ae6316d35.francecentral.azurecontainer.io/score


## Test du modèle

In [62]:
import json
input_payload = json.dumps({
    'data': ["realy ?","I love banana!","I hate banana!"],
    'method':"prediction_with_neutral"
})
output = service.run(input_payload)

print(output)

['neutral', 'good', 'bad']


In [ ]:
input_payload = json.dumps({
    'data': ["realy ?","I love banana!","I hate banana!"],
    'method':"score"
})
output = service.run(input_payload)

print(output)

In [ ]:
input_payload = json.dumps({
    'data': ["realy ?","I love banana!","I hate banana!"],
    'method':"prediction"
})
output = service.run(input_payload)

print(output)

## Suppression du service

Permet d'éviter les frais quand le service n'est plus "utile"

In [55]:
service.delete()